## Introduction to Data Ingestion

In [5]:
import os
from typing import List, Dict, Any
import pandas as pd

In [ ]:
from langchain_core.documents import Document
from langchain.text_splitter import (
    CharacterTextSplitter,
    RecursiveCharacterTextSplitter,
    TokenTextSplitter,
)

## Understanding document structure in Langchain

In [4]:
## reate a simple document
doc = Document(
    page_content="This will be the main content that will be embedded and seached",
    metadata={
        "source":"example.text",
        "page": 1
    }
)

print(f'Content: {doc.page_content}')
print(f'Metadata: {doc.metadata}')

Content: This will be the main content that will be embedded and seached
Metadata: {'source': 'example.text', 'page': 1}


In [6]:
type(doc)

langchain_core.documents.base.Document

### Reading text files

In [8]:
## Create a text file
os.makedirs("./data/text_files",exist_ok=True)

In [9]:
sample_texts={
    "./data/text_files/python_intro.txt":"""
    Python Programming Introduction

Python is a high-level, interpreted programming language known for its simplicity and readability.
Created by Guido van Rossum and first released in 1991, Python has become one of the most popular
programming languages in the world.

Key Features:
- Easy to learn and use
- Extensive standard library
- Cross-platform compatibility
- Strong community support

Python is widely used in web development, data science, artificial intelligence, and automation.
    """,
    "./data/text_files/machine_learning.txt": """
        Machine Learning Basics

Machine learning is a subset of artificial intelligence that enables systems to learn and improve
from experience without being explicitly programmed. It focuses on developing computer programs
that can access data and use it to learn for themselves.

Types of Machine Learning:
1. Supervised Learning: Learning with labeled data
2. Unsupervised Learning: Finding patterns in unlabeled data
3. Reinforcement Learning: Learning through rewards and penalties

Applications include image recognition, speech processing, and recommendation systems


    
    """
}

for filepath, content in sample_texts.items():
    with open(filepath,'w',encoding='utf-8') as f:
        f.write(content)
    f.close()

### TextLoader - Read Single File

In [10]:
from langchain.document_loaders import TextLoader

loader = TextLoader("./data/text_files/python_intro.txt", encoding='utf-8')

documents = loader.load()

print(documents)

[Document(metadata={'source': './data/text_files/python_intro.txt'}, page_content='\n    Python Programming Introduction\n\nPython is a high-level, interpreted programming language known for its simplicity and readability.\nCreated by Guido van Rossum and first released in 1991, Python has become one of the most popular\nprogramming languages in the world.\n\nKey Features:\n- Easy to learn and use\n- Extensive standard library\n- Cross-platform compatibility\n- Strong community support\n\nPython is widely used in web development, data science, artificial intelligence, and automation.\n    ')]


In [14]:
print(f'Loaded {len(documents)} documents')
print(f'Content preview:\n {documents[0].page_content[:100]}')
print(f'Metadata: {documents[0].metadata}')

Loaded 1 documents
Content preview:
 
    Python Programming Introduction

Python is a high-level, interpreted programming language known
Metadata: {'source': './data/text_files/python_intro.txt'}


### DirectoryLoader - Read Multiple Text Files

In [16]:
from langchain.document_loaders import DirectoryLoader

dir_loader = DirectoryLoader(
    path = './data/text_files',
    glob='**/*.txt', # Pattern to match files
    loader_cls=TextLoader, # Loader class to use
    loader_kwargs={'encoding': 'utf-8'}, # Arguments that the loader class expects
    show_progress=True,
)

documents = dir_loader.load()
for i, doc in enumerate(documents):
    print(f'Document: {i+1}')
    print(f"Source: {doc.metadata['source']}")
    print(f'Length: {len(doc.page_content)}')

100%|██████████| 2/2 [00:00<00:00, 57.74it/s]

Document: 1
Source: data/text_files/machine_learning.txt
Length: 585
Document: 2
Source: data/text_files/python_intro.txt
Length: 499


DirectoryLoader

Advantages:
- Loads multiple files at once
- Supports glob patterns
- Progress racking
- ecursive directory scanning

Disadvantages:
- All files must be of the same type
- Limited error handling per file
- Can be memory intensive for large directories

### Text Splitting Strategies

In [19]:
# Method 1 - Character Text Splitter
text = documents[0].page_content
char_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=200,
    chunk_overlap=20,
    length_function=len,
)

# Since we are passing a text object to the splitter, hence we use the method split_text
char_chunks = char_splitter.split_text(text)
print(f"Created {len(char_chunks)} chunks")
print(f"First chunk: {char_chunks[0][:100]}")

Created 4 chunks
First chunk: Machine Learning Basics
Machine learning is a subset of artificial intelligence that enables systems


In [21]:
for chunk_out in char_chunks:
    print("---------------")
    print(chunk_out)

---------------
Machine Learning Basics
Machine learning is a subset of artificial intelligence that enables systems to learn and improve
---------------
from experience without being explicitly programmed. It focuses on developing computer programs
that can access data and use it to learn for themselves.
Types of Machine Learning:
---------------
1. Supervised Learning: Learning with labeled data
2. Unsupervised Learning: Finding patterns in unlabeled data
3. Reinforcement Learning: Learning through rewards and penalties
---------------
Applications include image recognition, speech processing, and recommendation systems


Notice that we have added chunk_verlap = 20, however, we do not see that overlap in the output. This is because we have set the seperator = "\n" which leads to chunking at new lines.